## Modelling - ML

The goal for our work is to not only train a model to identify toxic comments, but to do so while reducing bias.
Bias in this task can be viewed as the situation where certain identies such as 'Black', 'Muslim', 'Gay' e.t.c, begin triggering toxic classification for comments they are in, even when the comment is actually positive. This is a key issue in toxic comment classification. 

The goal of the [jigsaw unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data) Kaggle challenge was to reduce this bias via a newly developed submetric which we have defined below.

**Note: The goal of the model is simply to predict the toxicity score of a model.** 

The bias weighted ROC metric below is calulated by taking segmenting the the dataset into identity subgroups by using the provided identity labels and then calculating the subgroup metrics.

In this notebook we have trained some baseline models which we will use as a baseline to compare our neural network against. 

### Metrics:

In addition to accuracy we will observe the below metrics for our models

#### Overall ROC-AUC:

This is the standard ROC-AUC for the full evaluation set. In other words this is the area under the Reciever Operating Characteristic curve. It compares the true positive and false positive rates of a binary model.

#### Subgroup ROC-AUC:

Here, we restrict the data set to only the examples that mention the specific identity subgroup. A low value in this metric means the model does a poor job of distinguishing between toxic and non-toxic comments that mention the identity.

#### BPSN AUC:

BPSN (Background Positive, Subgroup Negative) AUC: Here, we restrict the test set to the non-toxic examples that mention the identity and the toxic examples that do not. A low value in this metric means that the model confuses non-toxic examples that mention the identity with toxic examples that do not, likely meaning that the model predicts higher toxicity scores than it should for non-toxic examples mentioning the identity.

#### BNSP AUC:

BNSP (Background Negative, Subgroup Positive) AUC: Here, we restrict the test set to the toxic examples that mention the identity and the non-toxic examples that do not. A low value here means that the model confuses toxic examples that mention the identity with non-toxic examples that do not, likely meaning that the model predicts lower toxicity scores than it should for toxic examples mentioning the identity.


#### Generalized Mean of Bias AUCs
To combine the per-identity Bias AUCs into one overall measure, we calculate their generalized mean as defined below:

$M_p(m_s) = \left(\frac{1}{N} \sum_{s=1}^{N} m_s^p\right)^\frac{1}{p}$

Where:

$M_p$ = the $p$th power-mean function

$m_s$ = the bias metric $m$ calulated for subgroup $s$

$N$ = number of identity subgroups

For this competition, JigsawAI use a p value of -5 to encourage competitors to improve the model for the identity subgroups with the lowest model performance.

### Final Metric
We combine the overall AUC with the generalized mean of the Bias AUCs to calculate the final model score:

$score = w_0 AUC_{overall} + \sum_{a=1}^{A} w_a M_p(m_{s,a})$

$A$ = number of submetrics (3)

$m_{s,a}$ = bias metric for identity subgroup $s$ using submetric $a$

$w_a$ = $a$ weighting for the relative importance of each submetric; all four $w$ values set to 0.25


### Process:

#### Classical ML models
This is primarily an NLP task, our X feature matrix will be based off the text from online comments. We have defined a pre-processing pipeline in the 'preprocessing.ipynb' notebook to use for our our ML classifiers and a seperate pre-processing pipeline for the neural network models we are planning on training.

From the classic ML classifer models, we intend to use the following models - our base word embedding technique will be TF-IDF: 

   * Logistic Regression
   * SVM
   * Random Forest
 
   
We will carry out hyperparameter optimization for each model and calculate the metrics for each.

#### Neural Networks
*see NN_model.ipynb*

We will also train a neural network to answer this problem. We will start with a basic LSTM model which will be made of:
    
   * LSTM layers to read through the data
   * Dense layers
   * Output layer using sigmoid for the classes
   
We will then seek to improve this LSTM by creating a Bidrectional LSTM (BiLSTM) which we believe will improve accuracy by reading input sequences in both directions. If time allows we will also attempt to include a simple attention mechanism.

The NN models will use Glove 840B 300d word embeddings. 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import contractions
import tqdm
from tqdm import tqdm
tqdm.pandas()
import gc
import operator


import warnings
warnings.filterwarnings('ignore')

/Users/gov/anaconda3/envs/capstone/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Stage 1: Apply pre-processing to text

In [2]:
# Load train_data
train_df = pd.read_csv('data/train_clean.csv')

# Load in test data
test_df = pd.read_csv('data/test_clean.csv')

In [3]:
test_df.rename({'toxicity':'target'}, axis=1, inplace=1)

In [5]:
# Drop the unneeded columns
train_df = train_df.iloc[:,1:]
test_df = test_df.iloc[:,1:]

In [6]:
# In this cell we define the function that pre-processes our text

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import contractions
import string

def text_cleaner(df, col_name, clean_col_name):
    '''
    Text pre-processing pipeline, we lemmatize words, expand contractions, remove common stop words, apply lower case,
    tokenize, and delete punctuation. All functions use apply and list comprehension for speed benefit.
   
    INPUT:
    df = name of dataframe
    col_name = name of column to pre-process
    clean_col_name = name of new cleaned_column
   
    OUTPUT:
    None - changes are made directly to dataframe
    
    '''

    # Lemmatize helper functions
    # Lemmatize nouns
    def lemmatize_text_noun(text):
        return [lemmatizer.lemmatize(w, pos='n') for w in text]
    
    # Lemmatize verbs
    def lemmatize_text_verb(text):
        return [lemmatizer.lemmatize(w, pos='v') for w in text]
    # Lemmatize adjectives
    def lemmatize_text_adj(text):
        return [lemmatizer.lemmatize(w, pos='a') for w in text]

    # Lemmatize adverbs
    def lemmatize_text_adv(text):
        return [lemmatizer.lemmatize(w, pos='r') for w in text]
    
    # Expand contraction method
    def contraction_expand(text):
        return contractions.fix(text)
    
    # To lower case.
    df[clean_col_name] = df[col_name].apply(lambda x: x.lower())
    
    # Expand contractions
    df[clean_col_name] = df[clean_col_name].apply(lambda x: contraction_expand(x))
    
    #Tokenize:
    tokenizer = TweetTokenizer(reduce_len=True)
    df[clean_col_name] = df[clean_col_name].apply(lambda x: tokenizer.tokenize(x))
   
    
    #Remove Stop words
    stop_words = stopwords.words('english')
    df[clean_col_name] = df[clean_col_name].apply(lambda x: [item for item in x if item not in stop_words])
    
    #Delete punctuation
    punc_table = str.maketrans('', '', string.punctuation)
    df[clean_col_name] = df[clean_col_name].apply(lambda x: [item.translate(punc_table) for item in x])
    
    # LEMMATIZATION
    lemmatizer = WordNetLemmatizer()
    
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_noun)
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_verb)
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_adj)
    df[clean_col_name] = df[clean_col_name].apply(lemmatize_text_adv)
    
    
    return None


def detokenizer(df, col_name):
    detokenizer = TreebankWordDetokenizer()
    df[col_name+'_detokenize'] = df[col_name].apply(lambda x: detokenizer.detokenize(x))
    
    return None

In [7]:
%%time
# Run the cleaner func on train data
text_cleaner(train_df, 'comment_text', 'comment_text_clean')

CPU times: user 19min 18s, sys: 8.28 s, total: 19min 26s
Wall time: 19min 30s


In [ ]:
%%time 
# Run the same cleaner on the training data
text_cleaner(test_df, 'comment_text', 'comment_text_clean')

In [9]:
%%time
detokenizer(train_df,'comment_text_clean')

CPU times: user 3min 34s, sys: 803 ms, total: 3min 35s
Wall time: 3min 37s


In [10]:
%%time
# detokenize the test data
detokenizer(test_df, 'comment_text_clean')

CPU times: user 23.9 s, sys: 194 ms, total: 24.1 s
Wall time: 24.4 s


## Stage 2: Modelling

The data has been pre-processed for our models. We can now begin model training.

We have a seperate test dataset that will be kept aside for testing only once we have an ideal model. For hyperparameter optimizing we will use Scikit-learn's GridSearchCV. 

After we have fitted a model and predicted results, we can then append the predictions to the dataframe and calculate the subgroup AUCs and the final weighted metric . 

#### Defining subgroup AUC metrics

In [11]:
from sklearn import metrics
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive


# These calculations have been provided by Jigsaw AI for scoring based on the metrics of the kaggle competition
# https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview/evaluation

# They work by filtering the relevant dataframe into specific subgroups and using the roc_auc_score metric from sklearn.

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df.loc[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df.loc[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df.loc[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df.loc[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset.loc[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)



In [12]:
# These calculations have been provided by Jigsaw AI for scoring based on the metrics of the kaggle competition
# https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview/evaluation

# They work by filtering the relevant dataframe into specific subgroups and using the roc_auc_score metric from sklearn.

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    


y_train shape: (1804874,)
y_test shape: (194640,)


### Logistic Regression

We will use Grid Search Cross validation to optimize our hyperparameters on the training set. We will then fit the best estimator and then predict on the test set and calculate relevent metrics.

Because the subgroup ROCs are calculated post prediction and require predictions to be appended to a dataframe, we cannot actually pass the metrics into the scoring function of scikit-learn's gridsearchCV(). Instead we will just test the models on accuracy and total ROC-AUC and then refitting the grid-search on the model with the best ROC-AUC as a proxy.

In [19]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
#no scaling data is already transformed

# Instantiate the tokenizer to use in the vectorizer
tweet_tokenizer = TweetTokenizer(reduce_len=True)

# Instantiate the vectorizer, we will pass in the TweetTokenizer() from nltk 
tfid_vec_2 = TfidfVectorizer(lowercase=False, tokenizer = tweet_tokenizer.tokenize)

# define pipeline
pipeline = Pipeline([('tf-idf', tfid_vec_2), 
                     ('model', SVC())])



CPU times: user 1.65 ms, sys: 3.6 ms, total: 5.25 ms
Wall time: 8.3 ms


In [15]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
# Define scoring functions
scorers = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}


In [ ]:
# define parameters
param_grid_log = [{'model': [LogisticRegression()], 'tf-idf': [tfid_vec_2], 
                   'model__penalty': ['l1', 'l2'],
                   'model__C': [.001, 0.01, 0.1, 1, 10, 100]}]

# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score, i.e it finds the params that gives the best scores
# then refits using the ones that give the best AUC. 
grid_log = GridSearchCV(pipeline, param_grid_log, cv=5, scoring=scorers, refit='AUC', 
                        return_train_score=True, n_jobs=-1)

fittedgrid_log = grid_log.fit(train_df['comment_text_clean_detokenize'], train_df['target'])

In [ ]:
from sklearn.externals import joblib
# Save best estimator to file
joblib.dump(fittedgrid.best_estimator_, 'saved_models/best_log_reg.pkl')

In [50]:
#Run this to load the model if required
from sklearn.externals import joblib
from joblib import load
fittedgrid_log = load('saved_models/best_log_reg.pkl')

In [52]:
# Get scores
train_accuracy = fittedgrid_log.score(train_df['comment_text_clean_detokenize'], y_train)
test_accuracy = fittedgrid_log.score(test_df['comment_text_clean_detokenize'], y_test)

In [53]:
# predict train and test values
y_train_pred = fittedgrid_log.predict(train_df['comment_text_clean_detokenize'])
y_test_pred = fittedgrid_log.predict(test_df['comment_text_clean_detokenize'])

y_train_pred_prob = fittedgrid_log.predict_proba(train_df['comment_text_clean_detokenize'])
y_test_pred_prob = fittedgrid_log.predict_proba(test_df['comment_text_clean_detokenize'])

In [57]:
# Confusion Matrix and Classification Report, we want to store the F1 Score.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# store the precision, recall, and f1 score for later and print the classification report
log_precision = precision_score(y_test, y_test_pred)
log_recall = recall_score(y_test, y_test_pred)
log_f1 = f1_score(y_test, y_test_pred)

print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       0.96      0.99      0.97    179192
           1       0.76      0.50      0.60     15448

    accuracy                           0.95    194640
   macro avg       0.86      0.74      0.79    194640
weighted avg       0.94      0.95      0.94    194640



In [85]:
# Append preds and probabilities to train and valid dfs
train_df['Prediction_log'] = y_train_pred
train_df['Prediction_probability_log'] = y_train_pred_prob[:, 1]
test_df['Prediction_log'] = y_test_pred
test_df['Prediction_probability_log'] = y_test_pred_prob[:, 1]

In [34]:
# Identity columns used to calculate subgroup AUC
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']



In [ ]:
for col in identity_columns + ['target']:
    train_df[col] = np.where(train_df[col] >= 0.5, True, False)
    test_df[col] = np.where(test_df[col] >= 0.5, True, False)

In [ ]:
# Generate the AUC metrics 

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'

MODEL_NAME = 'Prediction_log'
TOXICITY_COLUMN = 'target'

log_bias_metrics_df_train = compute_bias_metrics_for_model(train_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
log_final_metric_train = get_final_metric(log_bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))

log_bias_metrics_df_test = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
log_final_metric_test = get_final_metric(log_bias_metrics_df_test, calculate_overall_auc(test_df, MODEL_NAME))

In [ ]:
log_bias_metrics_df_test

In [ ]:
print(f'train_accuracy:{train_accuracy}')
print(f'train weighted subgroup AUC:{get_final_metric(bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))}')
print(f'test_accuracy:{test_accuracy}')
print(f'test weighted subgroup AUC::{get_final_metric(bias_metrics_df_val, calculate_overall_auc(test_df, MODEL_NAME))}')


#### Model Evaluation:

The logistic regression performed well on pure accuracy, with a train accuracy of 95.38% and 94.96% test accuracy. What is also positive to see is that our hyperparameter optimization has led to a model which does not overfit excessively. 

However when we look at the weighted subgroup AUC metric, the 71.9% train score and 71.5% test score show that the model did have a tendency towards biased predictions for certain subgroup. In comparison, the benchmark CNN that was provided had a weighted AUC score of 88.35% albeit just on a validation set 


For example we can see that for the 'black' identity BPSN AUC was relatively low, suggesting the model is likely overweighting mentions of the 'black' identity with toxicity. 

--------------

### SVM

We will now carry out a similar process with SVM to see if this performs appreciably different to logistic regression. 

#### Grid searching on a subset of data

Given the length of time taken to grid search using SVM, we will run a gridsearch on a subset of our dataset by taking a new train test split. We will then use the results there as a proxy for the optimal parameters for our full dataset.  

In [ ]:
from sklearn.model_selection import train_test_split
# Create a new split of the training dataframe to use for this reduced test. 
# we will not need the remainder. We will take 1/4 of the total data set
remainder, reduced_df = train_test_split(train_df, test_size=0.25, stratify=train_df['target'], random_state=1)

In [ ]:
%%time
# Grid search applies tfid_vec_2 to all models. We try combinations of model loss function penalty strength and
# model gamma  is a parameter for non linear hyperplanes. 
# The higher the gamma value it tries to exactly fit the training data set

param_grid_svc = [{'model__kernel': ['rbf'],'tf-idf': [tfid_vec_2],
              'model__C': [0.1, 1, 10,], 'model__gamma': [1, 10, 'scale'], 'model__probability': [True]}]

reduced_grid_svm = GridSearchCV(pipeline, param_grid_svc, scoring=scorers, cv=3, refit='AUC',
                       return_train_score=True, n_jobs=-1, verbose=10)

reduced_grid_svm = reduced_grid_svm.fit(reduced_df['comment_text_clean_detokenize'], reduced_df['target'])

In [ ]:
# Save to file 
import joblib
joblib.dump(reduced_grid_svm.best_estimator_, 'saved_models/reduced_svm.pkl')

In [ ]:
# Gives the model with the best AUC score
reduced_grid_svm.best_estimator

In [ ]:
# gives the model with the best AUC score
reduced_grid_svm.best_score_

In [ ]:
%%time
# Full dataset grid search

param_grid_svc = [{'model__kernel': ['rbf'],'tf-idf': [tfid_vec_2],
              'model__C': [0.1, 1, 10], 'model__probability': [True]}]

grid_svc = GridSearchCV(pipeline, param_grid_svc, scoring=scorers, cv=5, refit='AUC',
                       return_train_score=True, n_jobs=-1, verbose=10)

fittedgrid_svc = grid_svc.fit(train_df['comment_text_clean_detokenize'], train_df['target'])

In [ ]:
# Save best estimator to file
joblib.dump(fittedgrid_svc.best_estimator_, 'saved_models/best_svm.pkl')

In [ ]:
# Get scores
train_accuracy = fittedgrid_svc.score(train_df['comment_text_clean_detokenize'], y_train)
test_accuracy = fittedgrid_svc.score(test_df['comment_text_clean_detokenize'], y_test)

In [ ]:
# predict train and test values
y_train_pred = fittedgrid_svc.predict(train_df['comment_text_clean_detokenize'])
y_test_pred = fittedgrid_svc.predict(test_df['comment_text_clean_detokenize'])

y_train_pred_prob = fittedgrid_svc.predict_proba(train_df['comment_text_clean_detokenize'])
y_test_pred_prob = fittedgrid_svc.predict_proba(test_df['comment_text_clean_detokenize'])

In [ ]:
# store the precision, recall, and f1 score for later and print the classification report
svm_precision = precision_score(y_test, y_test_pred)
svm_recall = recall_score(y_test, y_test_pred)
svm_f1 = f1_score(y_test, y_test_pred)

display(classification_report(y_test, y_test_pred))

In [84]:
# Append preds and probabilities to train and valid dfs


train_df['Prediction_svc'] = y_train_pred
train_df['Prediction_probability_svc'] = y_train_pred_prob[:, 1]
test_df['Prediction_svc'] = y_test_pred
test_df['Prediction_probability_svc'] = y_test_pred_prob[:, 1]

In [ ]:
# Generate the AUC metrics 

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'

MODEL_NAME = 'Prediction_svc'
TOXICITY_COLUMN = 'target'

svm_bias_metrics_df_train = compute_bias_metrics_for_model(train_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
svm_final_metric_train = get_final_metric(svm_bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))

svm_bias_metrics_df_test = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
svm_final_metric_test get_final_metric(svm_bias_metrics_df_test, calculate_overall_auc(test_df, MODEL_NAME))

--------

### Random Forest

For our final model we will try the Random Forest Classifier which is an ensemble method. It is based on the Decision Tree model, only the Random Forest works by fitting on random sub samples of the data (with replacement), which is known as 'bagging'. A voting algorithm is then applied on the results of each of the trees to determine the final class of the data point. The aim of Random Forest is to train a series of overfit models and then average out the results via voting to get better results. The main hyperparemeter we will optimize for is the number of decision trees to use. 


#### Grid Search on a subset of data

Given the length of time taken to grid search on the Random Forest Classifier, we will run a gridsearch on a subset of our dataset by taking a new train test split. We will then use the results there as a proxy for the optimal parameters for our full dataset. We will use the same train_test split as used for the SVM subset search. 

In [60]:
%%time
from sklearn.ensemble import RandomForestClassifier

# Note, most of the time here actually comes from the application of the tfid vectorizer to each fold of the data
# Random forest itself takes ~5minutes to run on the vectorized data.

# We control the model max depth 
param_grid_RF = {'model': [RandomForestClassifier()], 'tf-idf': [tfid_vec_2], 
                 'model__n_estimators': [10,50,100],
                 'model__max_depth': [100, 500, 1000, 5000],
             }
reduced_grid_RF = GridSearchCV(pipeline, param_grid_RF, scoring=scorers, cv=3, refit='AUC',
                       return_train_score=True, n_jobs=-1, verbose = 10)

reduced_grid_RF = reduced_grid_RF.fit(reduced_df['comment_text_clean_detokenize'], reduced_df['target'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 35.6min
[Parallel(n_jobs=-1)]: Done  25 out of  36 | elapsed: 78.8min remaining: 34.7min
[Parallel(n_jobs=-1)]: Done  29 out of  36 | elapsed: 113.6min remaining: 27.4min
[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed: 132.4min remaining: 12.0min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 165.0min finished


CPU times: user 50min 44s, sys: 12.9 s, total: 50min 57s
Wall time: 3h 35min 49s


In [61]:
# check the best estimator parameters
reduced_grid_RF.best_estimator_

Pipeline(memory=None,
         steps=[('tf-idf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=1000,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [62]:
# gives the best AUC score
reduced_grid_RF.best_score_

0.9230210872062684

In [ ]:
#reduced_grid_RF.cv_results_

The above test has suggested that in terms of hyperparameters, we should look at a model that has n_estimators around 100 or above, and max_depth of around 1000. We will use this as a guide for our grid search. 

#### Grid Search in the full dataset
Now that we have a good idea of optimal parameters, we will run another grid search on a few parameters around the optimal figures provided by our test on the subset of data. 

In [ ]:
# Running a grid-search on the full set of data 
param_grid_RF = {'model': [RandomForestClassifier()], 'tf-idf': [tfid_vec_2], 
                 'model__n_estimators': [100,150,200],
                 'model__max_depth': [1000, 2000, 3000, 4000],
             }
grid_RF = GridSearchCV(pipeline, param_grid_RF, scoring=scorers, cv=3, refit='AUC',
                       return_train_score=True, n_jobs=-1, verbose = 10)

fittedgrid_RF = grid_RF.fit(train_df['comment_text_clean_detokenize'], train_df['target'])

In [110]:
X_train = tfid_vec_2.transform(train_df['comment_text_clean_detokenize'])
X_test = tfid_vec_2.transform(test_df['comment_text_clean_detokenize'])
RF_model = RandomForestClassifier(n_estimators = 100, max_depth = 1000)
RF_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=1000, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [116]:
%%time
rf_test_accuracy = RF_model.score(X_test, y_test)


CPU times: user 1min 20s, sys: 3.61 s, total: 1min 24s
Wall time: 1min 26s


In [117]:
rf_test_accuracy

0.9417591450883682

In [119]:
# Save best estimator to file
joblib.dump(RF_model, 'saved_models/best_RF.pkl')

['saved_models/best_RF.pkl']

In [289]:
# predict train and test values
y_train_pred = RF_model.predict(X_train)
y_test_pred = RF_model.predict(X_test)

y_train_pred_prob = RF_model.predict_proba(X_train)
y_test_pred_prob = RF_model.predict_proba(X_test)

In [295]:
# store the precision, recall, and f1 score for later and print the classification report
rf_precision = precision_score(y_test, y_test_pred)
rf_recall = recall_score(y_test, y_test_pred)
rf_f1 = f1_score(y_test, y_test_pred)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97    179192
           1       0.83      0.34      0.48     15448

    accuracy                           0.94    194640
   macro avg       0.89      0.67      0.72    194640
weighted avg       0.94      0.94      0.93    194640



In [291]:
# Identity columns used to calculate subgroup AUC
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

In [292]:
for col in identity_columns + ['target']:
    train_df[col] = np.where(train_df[col] >= 0.5, True, False)
    test_df[col] = np.where(test_df[col] >= 0.5, True, False)

In [293]:
# Append preds and probabilities to train and valid dfs
train_df['Prediction_RF'] = y_train_pred
train_df['Prediction_probability_RF'] = y_train_pred_prob[:, 1]
test_df['Prediction_RF'] = y_test_pred
test_df['Prediction_probability_RF'] = y_test_pred_prob[:, 1]

In [294]:
# Generate the AUC metrics 

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'

MODEL_NAME = 'Prediction_RF'
TOXICITY_COLUMN = 'target'

rf_bias_metrics_df_train = compute_bias_metrics_for_model(train_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
rf_final_metric_train = get_final_metric(rf_bias_metrics_df_train, calculate_overall_auc(train_df, MODEL_NAME))

rf_bias_metrics_df_test = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
rf_final_metric_test = get_final_metric(rf_bias_metrics_df_test, calculate_overall_auc(test_df, MODEL_NAME))

In [296]:
rf_bias_metrics_df_test

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
6,black,1519,0.535805,0.671200,0.534692
3,christian,4226,0.538518,0.671294,0.536449
4,jewish,835,0.542044,0.668454,0.540397
5,muslim,2040,0.542836,0.672161,0.540382
2,homosexual_gay_or_lesbian,1065,0.546340,0.669129,0.545196
7,white,2452,0.556286,0.671977,0.555543
1,female,5155,0.573352,0.671357,0.572037
8,psychiatric_or_mental_illness,511,0.584684,0.664368,0.586552
0,male,4386,0.585907,0.670266,0.584965


In [297]:
rf_final_metric_test

0.6108032567341172

In [ ]:
print(f'train_accuracy:{train_accuracy}')
print(f'train weighted AUC:{rf_final_metric_train}')
print(f'test_accuracy:{test_accuracy}')
print(f'test weighted AUC::{rf_final_metric_test}')


Of the classical models the Random Forest was by far the weakest in terms of the final weighted AUC metric. It generally did poorly accross all the identity subgroups. In addition to this, it had very poor recall on the test set, implying that the model was failing to identify many cases of toxic commentary.

--------

### Model performance

Now that we have trained three seperate models which have been shown to deliver strong performance in text classification tasks in the past let us take the time to compare them side-by-side and also discuss their short-comings in terms of reducing bias

In [ ]:
#Display the subgroup bias metrics tables for each model and then the final metrics for each model for test only
display(log_bias_metrics_df_test)
display(svm_bias_metrics_df_test)
display(rf_bias_metrics_df_test)

In [ ]:
# Display final metrics for each model for test only.
print(f' Final metric for Logistic Regression: {log_final_metric_test}')
print(f' Final metric for SVM: {svm_final_metric_test}')
print(f' Final metric for Random Forest: {rf_final_metric_test}')

In [ ]:
data = {'Accuracy': [log_accuracy, svm_accuracy, rf_accuracy], 'Precision': [log_precision, svm_precision, rf_precision],
       'Recall': [log_recall, svm_recall, rf_recall], 'F1': [log_f1, svm_f1, rf_f1],
       'Final Bias Metric': [log_bias_metrics_df_test, svm_bias_metrics_df_test, rf_bias_metrics_df_test ]}

results_df = pd.DataFrame(data, index=['Logistic', 'Precision', 'Recall', 'Final Bias Metric'])

-----

## Exploring the results of our ML models using eli5

Below we use the eli5 package to help interpret our results. We will do this using the logistic model we fit due to only certain sklearn classifiers being compatible. 

In [66]:
import eli5
from eli5.lime import TextExplainer

Using TensorFlow backend.


In [175]:
from sklearn.linear_model import LogisticRegression
# We need to fit a standard Logistic Model using the same parametrs as our fitted grid as eli5 does not take in grids
eli5_log_model = LogisticRegression(C=1, penalty='l1')
# use the train and test transformed arrays from earlier
eli5_log_model.fit(X_train,y_train)

#score to check accuracy similar
accuracy = eli5_log_model.score(X_test, y_test)
#predict the test 
preds = eli5_log_model.predict_proba(X_test)

In [182]:
test_df['eli5_log'] = preds[:,-1]

In [183]:
test_df['eli5_log'] = np.where(test_df['eli5_log']>=0.5,1,0)

In [261]:
X_train

<1804874x181709 sparse matrix of type '<class 'numpy.float64'>'
	with 42012254 stored elements in Compressed Sparse Row format>

In [222]:
eli5.show_prediction(eli5_log_model, test_df.loc[23566,'comment_text_clean_detokenize'], vec=tfid_vec_2)

In [287]:
text = test_df.loc[92316,'comment_text']
text
eli5.explain_prediction(eli5_log_model, text, 
                        target_names=['Non-toxic', 'Toxic'], vec=tfid_vec_2)

Explanation(estimator="LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=100,\n                   multi_class='warn', n_jobs=None, penalty='l1',\n                   random_state=None, solver='warn', tol=0.0001, verbose=0,\n                   warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='Non-toxic', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=4.067819526352757, std=None, value=1.0), FeatureWeight(feature='preference', weight=0.15050612342487557, std=None, value=0.09848561374752673), FeatureWeight(feature='less', weight=0.036944886978590434, std=None, value=0.0500007178968633), FeatureWeight(feature='wonderful', weight=0.03366097949275505, std=None, value=0.04248781514865893), FeatureWeight(feature='part', weight=0.028071741398437572, std=None, value=0.028695163706261836), FeatureWeight(feature='typo', weight=0.026525265257097065, std=None, value=0.05552247344761501), FeatureWeight(feature='feel', weight=0.02510549275237288, std=None, value=0.030215840943073273), FeatureWeight(feature='question', weight=0.023841653295555622, std=None, value=0.02989906224280827), FeatureWeight(feature='condemnation', weight=0.02026188992118554, std=None, value=0.05234319218098195), FeatureWeight(feature='actually', weight=0.019820321467975, std=None, value=0.027909609531955874), FeatureWeight(feature='personal', weight=0.019710444406929213, std=None, value=0.03441866191382622), FeatureWeight(feature='business', weight=0.016951853666838285, std=None, value=0.02949374721939931), FeatureWeight(feature='play', weight=0.01457930155185926, std=None, value=0.031171737086051646), FeatureWeight(feature='think', weight=0.01441062144290144, std=None, value=0.02056907354815411), FeatureWeight(feature='do', weight=0.014166183433322727, std=None, value=0.029319737402930382), FeatureWeight(feature='life', weight=0.012448280707995136, std=None, value=0.027174048980096552), FeatureWeight(feature='fact', weight=0.010839712371861526, std=None, value=0.026469590531643414), FeatureWeight(feature='possible', weight=0.010553022429058853, std=None, value=0.035006343821040904), FeatureWeight(feature='family', weight=0.0093540006619206, std=None, value=0.029651813188166162), FeatureWeight(feature='know', weight=0.009237308450812094, std=None, value=0.02141555522935561), FeatureWeight(feature='support', weight=0.004999720265117958, std=None, value=0.027467486358471824), FeatureWeight(feature='young', weight=0.0036654504265455556, std=None, value=0.033846188512670905), FeatureWeight(feature='really', weight=0.00362754966135804, std=None, value=0.025857875671579034), FeatureWeight(feature='people', weight=0.0022696096266836482, std=None, value=0.01903367763501745)], neg=[FeatureWeight(feature='homosexual', weight=-1.5333567146966083, std=None, value=0.24161180220043157), FeatureWeight(feature='sexual', weight=-0.573117504249419, std=None, value=0.11510645269997512), FeatureWeight(feature='sex', weight=-0.22189934896406804, std=None, value=0.03863307178797715), FeatureWeight(feature='heterosexual', weight=-0.10525988881695267, std=None, value=0.05552247344761501), FeatureWeight(feature='wife', weight=-0.051797752776036335, std=None, value=0.03866720671384609), FeatureWeight(feature='men', weight=-0.03651617378313542, std=None, value=0.03383374101463431), FeatureWeight(feature='guy', weight=-0.03225704785716684, std=None, value=0.03004123887762275), FeatureWeight(feature='old', weight=-0.02868211338270907, std=None, value=0.029844398374451343), FeatureWeight(feature='I', weight=-0.016989373098195867, std=None, value=0.11251272723185926), FeatureWeight(feature='person', weight=-0.014201923335239518, std=None, value=0.02935669962375227), FeatureWeight(feature='absolutely', weight=-0.011204155808376842, std=None, value=0.03651218283971557), FeatureWeight(feature='care', weight=-0

In [217]:
text = test_df.loc[166392,'comment_text']
text

"Racist out comes don't have to motivated by a since of white superiority. Simple ignorance is all it takes. I worked in the public sector for 30 years in Eugene, often being the only black male in the office. I would not consider the vast majority of white folks I worked with to be conscious racists. However, the majority of them were ignorant about their own bias'. For example, they may say they hire people they feel comfortable with. However, since most of them never had a close black friend, they some how end up hiring only white folks. Of course, if they crew up in Idaho, how would they get to know any black people well enough to make friends? For example, I am the only black man to be a manager in the 100 year history of EWEB. For every one on those years EWEB was less than a mile from the UO. No one in management at EWEB think there is anything wrong or unusual with that reality. I would not be surprised if all public offices in Oregon have some degree of that blindness."

In [215]:
text2 = test_df.loc[23566,'comment_text_clean_detokenize']
text2

'racist come motivated since white superiority  simple ignorance take  worked public sector 30 year eugene  often black male office  would consider vast majority white folk worked conscious racist  however  majority ignorant bias   example  may say hire people feel comfortable  however  since never close black friend  end hiring white folk  course  crew idaho  would get know black people well enough make friend  example  black man manager 100 year history eweb  every one year eweb le mile uo  one management eweb think anything wrong unusual reality  would surprised public office oregon degree blindness'

In [206]:
test_df[(test_df['target'] == 0) & (test_df['eli5_log'] == 1) & 
        (test_df['black']==1)].iloc[0:30]

,id,comment_text,created_date,publication_id,article_id,rating,funny,wow,sad,likes,...,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,comment_text_clean,comment_text_clean_detokenize,Prediction_svc,Prediction_probability_svc,Prediction_log,Prediction_probability_log,eli5_log
878,7000878,"Oh man..., no one is trying to ""wear"" anyone d...",2017-08-28 16:42:44.817441+00,22,370137,approved,0,0,0,0,...,0,0,0,"[oh, man, , , one, trying, , wear, , anyone, l...",oh man one trying wear anyone left ask bl...,1,0.604803,1,0.604803,1
1090,7001090,Interesting that the DP failed to tell the ent...,2017-08-30 03:35:00.945664+00,102,372110,approved,0,0,0,17,...,0,0,0,"[interesting, dp, failed, tell, entire, story,...",interesting dp failed tell entire story sayin...,0,0.468953,0,0.468953,1
1329,7001329,"""Democrats renounced racism. ""\r\n\r\nSo Tim ...",2017-09-03 15:26:54.188281+00,13,373393,approved,0,0,0,0,...,0,0,0,"[, democrat, renounced, racism, , , tim, agree...",democrat renounced racism tim agrees democra...,0,0.427517,0,0.427517,1
1686,7001686,You can't make this stuff up ..\r\n.A white ra...,2017-10-12 02:26:36.040025+00,105,387829,approved,0,0,0,1,...,0,0,0,"[make, stuff, \r\n, white, rapper, rant, playe...",make stuff \r\n white rapper rant played black...,1,0.921078,1,0.921078,1
3562,7003562,What annoys me is the way these politically co...,2016-12-08 16:31:29.779533+00,54,155349,approved,2,0,0,2,...,0,0,0,"[annoys, way, politically, correct, term, gone...",annoys way politically correct term gone full ...,1,0.812175,1,0.812175,1
3599,7003599,You don't have a clue of what it's like to be ...,2017-02-25 15:16:06.873103+00,53,314382,approved,0,0,0,0,...,0,0,0,"[clue, like, , grew, among, black, , worked, b...",clue like grew among black worked black pla...,1,0.560420,1,0.560420,1
3602,7003602,"Well personally, as a white guy I was accused ...",2017-11-06 21:52:38.604522+00,102,397590,approved,0,0,0,0,...,0,0,0,"[well, personally, , white, guy, accused, raci...",well personally white guy accused racism furi...,1,0.939019,1,0.939019,1
4204,7004204,"Yes. \r\n\r\nUnlike women, gays, lesbians, tra...",2017-05-22 18:12:29.957359+00,13,336018,approved,1,0,0,2,...,0,0,0,"[yes, , unlike, woman, , gay, , lesbian, , tra...",yes unlike woman gay lesbian transgenders ...,1,0.860850,1,0.860850,1
4826,7004826,I'm talking about the eugenics practiced in Or...,2016-04-19 16:39:05.823989+00,13,60528,approved,0,0,0,0,...,0,0,0,"[I, talking, eugenics, practiced, oregon, invo...",I talking eugenics practiced oregon involuntar...,1,0.796792,1,0.796792,1
5844,7005844,It's simple.\r\nWorking means getting up and g...,2017-09-10 14:49:42.415101+00,102,376135,approved,1,0,1,3,...,0,0,0,"[simple, , working, mean, getting, going, work...",simple working mean getting going work buyin...,1,0.699297,1,0.699297,1


In [ ]:
# Load in log model
joblib.dump(fittedgrid.best_estimator_, 'saved_models/best_log_reg.pkl')

In [278]:
# Instantiate the lime TextExplainer model
text_model_log = TextExplainer(random_state=1)

#select a comment to investigate
text = test_df.loc[40972,'comment_text_clean_detokenize']

# Fit the Textexplainer using the logistic model predicted probabilites
text_model_log.fit(text, fittedgrid_log.predict_proba)
text_model_log.show_prediction(target_names=['Non-toxic', 'Toxic'])

In [ ]:
#92136 lgbt

In [233]:
test_df['comment_text'].values[2:3][0]

"No it won't . That's just wishful thinking on democrats fault .   For the 100 th time , Walker cited the cost of drug users treatment as being lost with Obamacare .  I laugh every time I hear a liberal claim republicans want to hurt people , and that's why they dumped Obamacare."

In [277]:
# Instantiate the lime TextExplainer model
text_model_log = TextExplainer(random_state=1)

#select a comment to investigate
text = test_df.loc[40972,'comment_text']

# Fit the Textexplainer using the logistic model predicted probabilites
text_model_log.fit(text,fittedgrid_log.predict_proba )
text_model_log.show_prediction(target_names=['Non-toxic', 'Toxic'])

In [272]:
test_df['comment_text'].values[87444]

'I\'m a liberal and I don\'t deny any of that. I despise Nathan Bedford Forrest. I despise Woodrow Wilson and his fawning over Birth of a Nation, which revitalized the KKK in the 1920s, including right here in the city Denver and the state of Colorado. I despise that Wilson waged a war to allegedly make the world "Safe for Democracy" while black Americans were being segregated and being terrorized by white supremacist rioters and lynch mobs at home. I despise the racist DixieCrats who threw a tantrum over Fanny Lou Hamer\'s Mississippi Freedom Democratic Party.\r\n\r\nThe difference between me and you is that I acknowledge these to purge them from my current party. You do it to deflect from the segregationist president and his white nationalist buddies that you cape for here every day.'

In [155]:
text2 = test_df.loc[92316,'comment_text']
text2
text3 = text2.replace("\n", "").replace("\r", "").replace("\\", '' )
text3 = " No, CE, I am not a homosexual.  I'm a monogamous heterosexual and have been married to my \
lively (was going to type 'lovely' and made a typo,  actually that 'lively' works too) wife, Sally, for 54 years. \
I have personal friends who ARE homosexual and have had those friendships for forty years or more.  \
They are absolutely wonderful people and the sexual preference doesn't play any role in our friendship. \
I really couldn't care less about anyone's sexual preference or sexual relationships.  \
I'm an 'old fashioned' guy and feel your sex life and romantic relationships are YOUR business. \
I do know of instances where the condemnation of family has destroyed the lives of \
young homosexual men and women, and it saddens me.  \
IMO, it's possible to support the rights and welfare of homosexual persons without being a homosexual yourself.  \
In fact, I think it's part of being a decent person.Hope that answers your question...Gary Cru "

In [280]:
text4 = test_df.loc[96458,'comment_text']
text4

'More news outlets are deleting their commentary sites; the great liberal bastion the Toronto Star and CNN dropped theirs ages ago as they could not take criticism. Other outlets increasingly  only allow comments on safe topics. Others censor or ban anyone who upsets the apple cart by talking about sacred cows or elephants in the room: such as homosexuals and aids, or black crime, or overpaid civil servants,  Canadian Jews and Israel, or anything deemed politically incorrect. Of course it is okay to bash white Anglo saxon men or Muslims. But it is not okay to point out the  striking similarities between Orthodox Jews and Muslims and how they treat their women, or how a lot of Muslim men chase Christian and Jewish girls but we cant go near their girls. And it is not okay to talk about how only lower middle class Christian boys fight and die for Canada in te middle east wars while so many others hide out in elite schools. I wish some academics would address these issues objectively.'

In [279]:
test_df[(test_df['target'] == 0) & (test_df['Prediction_log'] == 1) & (test_df['black']==1)].iloc[60:70]

,id,comment_text,created_date,publication_id,article_id,rating,funny,wow,sad,likes,...,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,comment_text_clean,comment_text_clean_detokenize,Prediction_svc,Prediction_probability_svc,Prediction_log,Prediction_probability_log,eli5_log
91787,7091787,set race relations back by governing while bla...,2017-01-03 00:20:51.418615+00,54,159664,approved,0,0,0,2,...,0,0,0,"[set, race, relation, back, governing, black, ...",set race relation back governing black wonder...,1,0.785751,1,0.785751,1
91923,7091923,I don't know the background to this case excep...,2017-08-18 18:56:37.782689+00,100,367936,approved,0,0,0,0,...,0,0,0,"[know, background, case, except, white, extrem...",know background case except white extremist ki...,1,0.909707,1,0.909707,1
92740,7092740,>I am not defending the DUP\r\nLiar ! You STAR...,2017-06-10 06:40:45.143685+00,100,342372,rejected,0,0,0,0,...,0,0,0,"[, defending, dup, liar, , started, thread, de...",defending dup liar started thread defending d...,1,0.524313,1,0.524313,1
93236,7093236,We bring flowers to the dead be they school ki...,2016-06-24 20:09:07.725714+00,22,139610,approved,0,0,0,1,...,0,0,0,"[bring, flower, dead, school, kid, , black, bi...",bring flower dead school kid black bible stud...,1,0.514338,1,0.514338,1
96103,7096103,Mr. Harris is not a liberal. He is just an edu...,2017-06-08 21:25:51.347801+00,13,341755,approved,0,0,0,1,...,0,0,0,"[mr, , harris, liberal, , educated, black, man...",mr harris liberal educated black man suspec...,1,0.516916,1,0.516916,1
96458,7096458,More news outlets are deleting their commentar...,2017-04-27 13:48:21.733011+00,54,329864,approved,0,0,0,6,...,0,0,0,"[news, outlet, deleting, commentary, site, , g...",news outlet deleting commentary site great li...,1,0.538284,1,0.538284,1
97108,7097108,"""Buckwheat"" was a Black character in the Littl...",2016-10-28 20:02:04.557455+00,21,149818,approved,0,0,0,2,...,0,0,0,"[, buckwheat, , black, character, little, rasc...",buckwheat black character little rascal movie...,1,0.652036,1,0.652036,1
97291,7097291,There's opinions and there's lies . Shannyn pr...,2016-10-16 17:10:27.898600+00,21,148649,approved,0,0,0,2,...,0,0,0,"[opinion, lie, , shannyn, probably, know, diff...",opinion lie shannyn probably know difference ...,1,0.746986,1,0.746986,1
98787,7098787,Why because police show up after the crime? Or...,2017-11-10 02:55:35.648452+00,102,399117,approved,0,1,0,1,...,0,0,0,"[police, show, crime, , troll, poor, neighborh...",police show crime troll poor neighborhood har...,1,0.575684,1,0.575684,1
103100,7103100,3/ << From African-American activist Angela Da...,2017-04-28 01:32:45.850482+00,53,328923,approved,0,0,0,0,...,0,0,0,"[3, , , , africanamerican, activist, angela, d...",3 africanamerican activist angela davis lef...,1,0.806735,1,0.806735,1
